In [2]:
import heapq
import geopy.distance
import math
import csv
import Utils
import time
  
PUBLIC_TRANSPORT_SPEED_KM_H = 10

class Graph:
    def __init__(self, edges):
        self.edges = edges
        self.graph_dict = {}
        for start, end, line, start_time, arrival_time, weight, start_stop_lat, start_stop_lon, end_stop_lat, end_stop_lon in self.edges:
            if start in self.graph_dict:
                self.graph_dict[start].append((end, line, start_time, arrival_time, weight, start_stop_lat, start_stop_lon, end_stop_lat, end_stop_lon))
            else:
                self.graph_dict[start] = [(end, line, start_time, arrival_time, weight, start_stop_lat, start_stop_lon, end_stop_lat, end_stop_lon)]
            if end not in self.graph_dict:
                self.graph_dict[end] = []

class LineGraph:
    def __init__(self, edges):
        self.edges = edges
        self.graph_dict = {}
        for _, end, line, _, arrival_time, _, _, _, _, _ in self.edges:
            if end in self.graph_dict:
                self.graph_dict[end].add(line)
            else:
                self.graph_dict[end] = {line}

class LineAndTimeGraph:
    def __init__(self, edges):
        self.edges = edges
        self.graph_dict = {}
        for _, end, line, _, arrival_time, _, _, _, _, _ in self.edges:
            if (end, line) in self.graph_dict:
                self.graph_dict[(end, line)].add(arrival_time)
            else:
                self.graph_dict[(end, line)] = {arrival_time}
            
def edges(fileName):
    edges = []
    with open(fileName, 'r', encoding='utf8') as file:
        reader = csv.reader(file)
        header = next(reader)
        
        for row in reader:
            line = row[2]
            start_time = Utils.minutes_from_midnight(row[3])
            arrival_time = Utils.minutes_from_midnight(row[4]) 
            weight = arrival_time - start_time
            
            if weight < 0:
                weight += 24*60
            
            start = row[5]
            end = row[6]
            
            start_stop_lat = row[7]
            start_stop_lon = row[8]
            end_stop_lat = row[9]
            end_stop_lon = row[10]
            
            edges.append((start, end, line, start_time, arrival_time, weight, start_stop_lat, start_stop_lon, end_stop_lat, end_stop_lon))
    return edges


def distance_from_coords(coords1, coords2):
    return geopy.distance.geodesic(coords1, coords2).km

def time_from_distance(distance):
    return distance/PUBLIC_TRANSPORT_SPEED_KM_H * 60
    
def time_from_coords(coords1, coords2):
    return time_from_distance(distance_from_coords(coords1, coords2))


In [3]:
edges = edges('connection_graph_fixed.csv')
gg = Graph(edges)
lg = LineGraph(edges)
lng = LineAndTimeGraph(edges)

In [4]:
def get_best_direct_line(graph, goal_lines, line_n_time_graph, start_time, optimistic_arrival_time, goal, start):
    fastest_direct_time = 25*60
    best_line = None
    lines_visited = []
    for _, line, _, _, _, _, _, _, _ in graph[start]:
        if line in lines_visited:
            continue
        else:
            lines_visited.append(line)
            
        if line in goal_lines:
            best_line_time = 25*60
            for arrival_time in line_n_time_graph[goal, line]:
                if arrival_time >= optimistic_arrival_time:
                    if arrival_time - start_time < best_line_time:
                        best_line_time = arrival_time - start_time
                elif arrival_time + 24*60 - start_time < best_line_time:
                    best_line_time = arrival_time + 24*60 - start_time
            if best_line_time < fastest_direct_time:
                fastest_direct_time = best_line_time
                best_line = line
    return best_line, fastest_direct_time

def get_first_direct_line(graph, goal_lines, start_time, start):
    min_direct_wait_time = 25*60
    best_line = None
    curr_line = None
    for _, line, line_start_time, _, _, _, _, _, _ in graph[start]:
        if curr_line != line:
            min_wait_time = 25*60
            curr_line = line
            
        if line in goal_lines:
            if line_start_time >= start_time:
                if line_start_time - start_time < min_wait_time:
                    min_wait_time =  line_start_time - start_time
            elif line_start_time + 24*60 - start_time < min_wait_time:
                min_wait_time =  line_start_time + 24*60 - start_time
        
            if min_wait_time < min_direct_wait_time:
                min_direct_wait_time = min_wait_time
                best_line = line          

    return best_line, min_direct_wait_time

In [5]:
LINE_CHANGE_COST = 20

def astar_p_old(graph_dict, start, goal, trip_start_time, coords_heuristic_fn):
    goal_info = graph_dict[goal][0]
    goal_coords = (goal_info[5], goal_info[6])
    
    start_info = graph_dict[start][0]
    current_coords = (start_info[5], start_info[6])
    
    counter = 0
    
    pq = [(0, start)]
    prev_nodes = {start: None}
    distances = {node: float('inf') for node in graph_dict}
    distances[start] = 0
    
    fScore = {node: float('inf') for node in graph_dict}
    fScore[start] = 0
    
    trip_start_time = Utils.minutes_from_midnight(trip_start_time)

    while pq:
        _, current = heapq.heappop(pq)
        curr_cost = distances[current]
        curr_f_score = fScore[current]
        curr_arrival_time = trip_start_time + curr_cost
        base_days_en_route = 0
        if current == goal:
            break
        
        if curr_arrival_time > 24*60:
            base_days_en_route = int(curr_arrival_time/(24*60))
            curr_arrival_time = curr_arrival_time%(24*60)
            
        prev_line = ''
        if prev_nodes[current]:
            prev_line = prev_nodes[current][1]
            
        for neighbor, line, start_time, arrival_time, weight, start_stop_lat, start_stop_lon, _, _ in graph_dict[current]:
            
            if prev_line == line or prev_line == '': line_change_penalty = 0
            else: line_change_penalty = LINE_CHANGE_COST 
                
            counter += 1
            days_en_route = base_days_en_route
            current_coords = (start_stop_lat, start_stop_lon)

            if curr_arrival_time <= start_time:
                # current travel time + time spent waiting for the bus/tram + time spent en route 
                new_cost = curr_cost + (start_time - (curr_arrival_time)) + weight
            else:
                # current travel time + (time till midnight + start_time) + time spent en route
                new_cost = curr_cost + (24*60 - (curr_arrival_time) + start_time) + weight
                days_en_route += 1
            
            if curr_f_score + new_cost + line_change_penalty < fScore[neighbor]:
                distances[neighbor] = new_cost
                fScore[neighbor] = curr_f_score + new_cost + line_change_penalty
                priority = curr_f_score + new_cost + math.floor(coords_heuristic_fn(current_coords, goal_coords)) + line_change_penalty
                # priority = new_cost
                heapq.heappush(pq, (priority, neighbor))
                prev_nodes[neighbor] = (current, line, start_time + days_en_route*24*60, arrival_time + days_en_route*24*60)

    print(f'COUNTER: {counter}')
    return distances, prev_nodes

def astar_p(graph_dict, lines_dict, line_n_time_graph, start, goal, trip_start_time, coords_heuristic_fn):
    goal_info = graph_dict[goal][0]
    goal_coords = (goal_info[5], goal_info[6])
    
    start_info = graph_dict[start][0]
    current_coords = (start_info[5], start_info[6])
    
    direct_line = None
    counter = 0
    
    pq = [(0, start)]
    prev_nodes = {start: None}
    distances = {node: float('inf') for node in graph_dict}
    distances[start] = 0
    
    fScore = {node: float('inf') for node in graph_dict}
    fScore[start] = 0
    
    trip_start_time = Utils.minutes_from_midnight(trip_start_time)

    while pq:
        _, current = heapq.heappop(pq)
        curr_cost = distances[current]
        curr_f_score = fScore[current]
        curr_arrival_time = trip_start_time + curr_cost
        base_days_en_route = 0
        if current == goal:
            break
        
        if curr_arrival_time > 24*60:
            base_days_en_route = int(curr_arrival_time/(24*60))
            curr_arrival_time = curr_arrival_time%(24*60)
            
        prev_line = ''
        if prev_nodes[current]:
            prev_line = prev_nodes[current][1]
            
        for neighbor, line, start_time, arrival_time, weight, start_stop_lat, start_stop_lon, _, _ in graph_dict[current]:
                
            # if there is a direct connection, but this aint it
            if direct_line:
                if line != direct_line:
                    continue
            elif line in lines_dict[goal]:       
                optimistic_arrival_time = curr_arrival_time + math.floor(coords_heuristic_fn(current_coords, goal_coords))
                # direct_line, best_time = get_best_direct_line(graph_dict, lines_dict[goal], line_n_time_graph, curr_arrival_time, optimistic_arrival_time, goal, current)
                # if best_time > (optimistic_arrival_time - curr_arrival_time) * 3:
                #     direct_line = None
                direct_line, best_time = get_first_direct_line(graph_dict, lines_dict[goal], curr_arrival_time, current)
                if best_time > LINE_CHANGE_COST:
                    direct_line = None
                if line != direct_line:
                    continue
                
            if prev_line == line or prev_line == '': line_change_penalty = 0
            else: line_change_penalty = LINE_CHANGE_COST 
                
            counter += 1
            days_en_route = base_days_en_route
            current_coords = (start_stop_lat, start_stop_lon)

            if curr_arrival_time <= start_time:
                # current travel time + time spent waiting for the bus/tram + time spent en route 
                new_cost = curr_cost + (start_time - (curr_arrival_time)) + weight
            else:
                # current travel time + (time till midnight + start_time) + time spent en route
                new_cost = curr_cost + (24*60 - (curr_arrival_time) + start_time) + weight
                days_en_route += 1
            
            if curr_f_score + new_cost + line_change_penalty < fScore[neighbor]:
                distances[neighbor] = new_cost
                fScore[neighbor] = curr_f_score + new_cost + line_change_penalty
                priority = curr_f_score + new_cost + math.floor(coords_heuristic_fn(current_coords, goal_coords)) + line_change_penalty

                heapq.heappush(pq, (priority, neighbor))
                prev_nodes[neighbor] = (current, line, start_time + days_en_route*24*60, arrival_time + days_en_route*24*60)

    print(f'COUNTER: {counter}')
    return distances, prev_nodes

In [6]:
def shortest_path_p_old(graph_dict, start, goal, trip_start_time, coords_heuristic_fn):
    calc_start_time = time.time()
    trip_length, prev_nodes, prev_line = astar_p(graph_dict, start, goal, trip_start_time, coords_heuristic_fn)
    calc_end_time = time.time()
    print(f'time_taken: {round(calc_end_time - calc_start_time, 3)}s')
    path = []
    line = ''
    curr_node = goal
    start_time = 0
    arrival_time = Utils.minutes_from_midnight(trip_start_time) + trip_length
    while True:
        path.append((curr_node, line, start_time, arrival_time))
        if(prev_nodes[(curr_node, prev_line)]):
            line = prev_line
            curr_node, prev_line, start_time, arrival_time = prev_nodes[(curr_node, line)]

        else:
            break
    path.reverse()
    Utils.print_solution(trip_start_time, trip_length, path)

def shortest_path_p_old_2(graph_dict, start, goal, trip_start_time, coords_heuristic_fn):
    calc_start_time = time.time()
    distances, prev_nodes = astar_p(graph_dict, start, goal, trip_start_time, coords_heuristic_fn)
    calc_end_time = time.time()
    print(f'time_taken: {round(calc_end_time - calc_start_time, 3)}s')
    path = []
    curr_node = goal
    line = ''
    trip_length = distances[goal]
    start_time = 0
    arrival_time = Utils.minutes_from_midnight(trip_start_time) + trip_length
    while True:
        path.append((curr_node, line, start_time, arrival_time))
        if(prev_nodes[curr_node]):
            curr_node, line, start_time, arrival_time = prev_nodes[curr_node]
        else:
            break
    path.reverse()
    Utils.print_solution(trip_start_time, trip_length, path)
    
def shortest_path_p(graph_dict, lines_dict, line_n_time_graph, start, goal, trip_start_time, coords_heuristic_fn):
    calc_start_time = time.time()
    distances, prev_nodes = astar_p(graph_dict, lines_dict, line_n_time_graph, start, goal, trip_start_time, coords_heuristic_fn)
    calc_end_time = time.time()
    print(f'time_taken: {round(calc_end_time - calc_start_time, 3)}s')
    path = []
    curr_node = goal
    line = ''
    trip_length = distances[goal]
    start_time = 0
    arrival_time = Utils.minutes_from_midnight(trip_start_time) + trip_length
    while True:
        path.append((curr_node, line, start_time, arrival_time))
        if(prev_nodes[curr_node]):
            curr_node, line, start_time, arrival_time = prev_nodes[curr_node]
        else:
            break
    path.reverse()
    Utils.print_solution(trip_start_time, trip_length, path)

In [7]:
# shortest_path_p(gg.graph_dict, lg.graph_dict, lng.graph_dict,'pl. Bema', 'DWORZEC GŁÓWNY', '12:00:00', time_from_coords)
# shortest_path_p(gg.graph_dict, lg.graph_dict, lng.graph_dict,'Ogród Botaniczny', 'Rynek', '20:00:00', time_from_coords)
# shortest_path_p(gg.graph_dict, lg.graph_dict, lng.graph_dict,'Białowieska', 'most Grunwaldzki', '12:00:00', time_from_coords)
# shortest_path_p(gg.graph_dict, lg.graph_dict, lng.graph_dict,'Kwiska', 'PL. GRUNWALDZKI', '09:00:00', time_from_coords)
# shortest_path_p(gg.graph_dict, lg.graph_dict, lng.graph_dict,'Iwiny - rondo', 'Hala Stulecia', '14:07:00', time_from_coords)
# shortest_path_p(gg.graph_dict, lg.graph_dict, lng.graph_dict,'Krzemieniecka', 'Prusa', '20:07:00', time_from_coords)
# shortest_path_p(gg.graph_dict, lg.graph_dict, lng.graph_dict,'KOSZAROWA (Szpital)', 'Waniliowa', '07:07:00', time_from_coords)
# shortest_path_p(gg.graph_dict, lg.graph_dict, lng.graph_dict,'Ossolineum (Uniwersytecka)', 'Kasprowicza', '23:07:00', time_from_coords)
# shortest_path_p(gg.graph_dict, lg.graph_dict, lng.graph_dict,'Na Ostatnim Groszu', 'Biegasa', '9:07:00', time_from_coords)
# shortest_path_p(gg.graph_dict, lg.graph_dict, lng.graph_dict,'Orla', 'pl. Strzegomski (Muzeum Współczesne)', '21:07:00', time_from_coords)
# shortest_path_p(gg.graph_dict, lg.graph_dict, lng.graph_dict,'Damrota', 'GIEŁDOWA (Centrum Hurtu)', '00:12:00', time_from_coords)
# shortest_path_p(gg.graph_dict, lg.graph_dict, lng.graph_dict,'Łozina - Wrocławska (na wys. nr 18)', 'Chopina', '10:07:00', time_from_coords)
shortest_path_p(gg.graph_dict, lg.graph_dict, lng.graph_dict,'Magellana', 'DWORZEC AUTOBUSOWY', '10:07:00', time_from_coords)


COUNTER: 25282
time_taken: 0.134s
time: 00:34:00
start time: 10:07:00
in: Magellana [10:08:00]| line: 941
out: SĘPOLNO [10:12:00]| line: 941
in: SĘPOLNO [10:13:00]| line: 9
out: DWORZEC AUTOBUSOWY [10:41:00]| line: 9
-
-
-
